<a href="https://colab.research.google.com/github/Anjana2002/Language-Identification-for-Malayalam-English-Code-Mixed-Text/blob/main/data%20annotation/annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
# def remove_emoji(text):
#     if isinstance(text, str):
#         emoji_pattern = re.compile(
#             "["
#             u"\U0001F600-\U0001F64F"  # Emoticons
#             u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
#             u"\U0001F680-\U0001F6FF"  # Transport & map symbols
#             u"\U0001F700-\U0001F77F"  # Alchemical symbols
#             u"\U0001F780-\U0001F7FF"  # Geometric shapes extended
#             u"\U0001F800-\U0001F8FF"  # Supplemental arrows-C
#             u"\U0001F900-\U0001F9FF"  # Supplemental symbols and pictographs
#             u"\U0001FA00-\U0001FA6F"  # Chess symbols
#             u"\U0001FA70-\U0001FAFF"  # Symbols and pictographs extended-A
#             u"\U00002702-\U000027B0"  # Dingbats
#             u"\U000024C2-\U0001F251"  # Enclosed characters
#             u"\U0001F1E0-\U0001F1FF"  # Flags (iOS)
#             "]+", flags=re.UNICODE)

#         text = emoji_pattern.sub(r'', text)
# #         return text.strip() if text.strip() else None  # Return None if text is empty after removing emojis
# #     return text
# # df = df.applymap(remove_emoji)
# # def remove_numbers(text):
# #     if isinstance(text, str):
# #         text = re.sub(r'[!@#$%^&*()_\-+=\[\]{}|\\;:"\'<>,./?`~]', '', text)
# #         text = re.sub(r'\d+', '', text)
# #         text = text.strip()
# #         return text if text else None
# #     return text
# # df = df.applymap(remove_numbers)
# # def remove_non_visible(text):
# #     if isinstance(text, str):
# #         # This removes zero-width spaces and other invisible characters
# #         return re.sub(r'[^\x20-\x7E]+', '', text).strip() or None
# #     return text
# # df = df.applymap(remove_non_visible)
# # df.head()
# def tokenize_text(text):
#   return word_tokenize(text)
# df['text'] = df['text'].apply(tokenize_text)
# print(df.head(10))
# lemmatizer = WordNetLemmatizer()

# def pos_and_lemmatize(text):
#   pos_tags = pos_tag(text)
#   lemmatized_results = []
#   for word, tag in pos_tags:
#     if tag.startswith('NN'):
#       pos = 'n'
#     elif tag.startswith('VB'):
#       pos = 'v'
#     elif tag.startswith('JJ'):
#       pos = 'a'
#     elif tag.startswith('RB'):
#       pos = 'r'
#     else:
#       pos = 'n'
#     lemma = lemmatizer.lemmatize(word, pos)
#     lemmatized_results.append({'word': word, 'lemma': lemma, 'pos': tag})

#   return lemmatized_results
# df['pos_lemma'] = df['text'].apply(pos_and_lemmatize)
# print(df[['text', 'pos_lemma']])
# english_words = set(nltk_words.words())
# stop_words = set(stopwords.words('english'))
# import nltk
# from nltk.tokenize import word_tokenize, sent_tokenize
# from nltk import pos_tag
# from nltk.corpus import words as nltk_words, stopwords
# from nltk.stem import WordNetLemmatizer
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
# nltk.download('words')
# nltk.download('wordnet')
# !pip install pandas nltk

## DATA ANNOTATION

In [71]:
!pip install pandas nltk

In [72]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from nltk.corpus import words as nltk_words, stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
df = pd.read_csv("malayalam_youtube_comments_filtered.csv")

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [73]:
english_word_set = set(nltk_words.words())

stop_words_set = set(stopwords.words('english'))
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # Emoticons
                           u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # Transport & map symbols
                           u"\U0001F700-\U0001F77F"  # Alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric shapes extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental symbols and pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and pictographs extended-A
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251"  # Enclosed characters
                           u"\U0001F1E0-\U0001F1FF"  # Flags
                           "]+", flags=re.UNICODE)

In [74]:
acronym_list = {"NASA", "AI", "ML", "COVID", "USA"}  # Example set of common acronyms
universal_symbols = {"#", "@", "&", "%", "*"}
malayalam_pattern = re.compile(r'[\u0D00-\u0D7F]+')

In [75]:
lemmatizer = WordNetLemmatizer()

In [76]:
def word_length(word):
    return len(word)

def is_mixed(word):
    # Check if the word has both English and Malayalam characters
    return bool(re.search(r'[a-zA-Z]', word)) and bool(malayalam_pattern.search(word))

def lemmatize_word(word, tag):
    # Map POS tag to lemmatizer format
    tag = tag[0].lower()  # Convert NLTK tag to first letter format
    if tag == 'n':
        pos = 'n'
    elif tag == 'v':
        pos = 'v'
    elif tag == 'a':
        pos = 'a'
    elif tag == 'r':
        pos = 'r'
    else:
        pos = 'n'  # Default to noun if not recognized
    return lemmatizer.lemmatize(word, pos)



In [77]:
def tag_comment(comment):
    temp_comments = []
    words = word_tokenize(comment)  # Tokenize the comment

    # POS tagging and lemmatization
    pos_tags = pos_tag(words)
    lemmatized_words = [(lemmatize_word(word, tag), tag) for word, tag in pos_tags]

    for word, tag in lemmatized_words:
        if word in english_words_set:
            temp_comments.append((word, "eng"))
        elif emoji_pattern.match(word):
            temp_comments.append((word, "univ"))
        elif word in stop_words_set:
            temp_comments.append((word, "eng"))
        elif word in acronym_list:
            temp_comments.append((word, "acr"))
        elif word in universal_symbols:
            temp_comments.append((word, "univ"))
        elif word_length(word) < 3:
            temp_comments.append((word, "undef"))
        elif word_length(word) > 3 and is_mixed(word):
            temp_comments.append((word, "mix"))
        elif malayalam_pattern.search(word):
            temp_comments.append((word, "mal"))
        else:
            temp_comments.append((word, "undef"))

    return temp_comments

In [78]:
df['annotated'] = df['text'].apply(tag_comment)
df.to_csv('mala.csv', index=False)
df.head()

,video_id,text,annotated
0,63i4ZQcvpcQ,Download Duolingo for free: https://app.adjust...,"[(Download, undef), (Duolingo, undef), (for, e..."
1,63i4ZQcvpcQ,Nayanthara kettiyathin shesham kanunnavar,"[(Nayanthara, undef), (kettiyathin, undef), (s..."
2,63i4ZQcvpcQ,00:12 which is that song playing in the backgr...,"[(00:12, undef), (which, eng), (be, eng), (tha..."
3,63i4ZQcvpcQ,2024 july 21n kaanunna njan🥲🥲,"[(2024, undef), (july, undef), (21n, undef), (..."
4,63i4ZQcvpcQ,15:57 what she is saying,"[(15:57, undef), (what, eng), (she, eng), (be,..."
